In [23]:
#Necessary imports

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import pandas as pd
import dlib
import seaborn as sns


from scipy.signal import butter, filtfilt, lfilter
from scipy.fft import fft, fftfreq
from datetime import datetime

In [24]:
#Constants
low_cutoff = 0.75
high_cutoff = 2.5
fps=0
fs = 2 * high_cutoff

#Paths
dlib_path = 'dlib_files/shape_predictor_68_face_landmarks.dat'
# Load face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dlib_path)


#pulses values
pulses_forehead=[]
pulses_leftc=[]
pulses_rightc=[]
# pulses_fullf=[]

# Create a new DataFrame with new columns data


In [25]:
#Utility functions

# def setValues():
#     #Constants
#     low_cutoff = 0.75
#     high_cutoff = 2.5
#     fs = 2 * high_cutoff    
#     #pulses values
#     # pulses_fullf=[]


def calculate_pulse_signal(roi,pulses):
    
    B, G, R = cv2.split(roi)
    
    #Normalizing R,G,B channels
    
    Rn = R / np.mean(R)
    Gn = G / np.mean(G)
    Bn = B / np.mean(B)    
    
    #CHROM Signals
    
    Xs = 3*Rn - 2*Gn
    Ys = 1.5*Rn + Gn - 1.5*Bn
    
    Rf = apply_bandpass_filter(Rn)
    Gf = apply_bandpass_filter(Gn)
    Bf = apply_bandpass_filter(Bn)
    Xf = apply_bandpass_filter(Xs)
    Yf = apply_bandpass_filter(Ys)

    alpha = np.std(Xf) / np.std(Yf)

    Signal = 3*(1-(alpha/2))*Rf - 2*(1 + (alpha/2))*Gf + ((3*alpha)/2)*Bf
    
    pulses.append(np.mean(Signal))  

def apply_bandpass_filter(channel, order=5):
    b, a = butter_bandpass(order)
    y = filtfilt(b, a, channel, axis=0,padlen=0)
    return y

def butter_bandpass(order=5):
    nyquist = 0.5 * fs  # Nyquist frequency
    lowcut_norm = low_cutoff / nyquist 
    highcut_norm = high_cutoff / nyquist
    b, a = butter(order, [lowcut_norm, highcut_norm-0.1], btype='band')
    return b, a

    

In [26]:
# Open video file

def calculateResult(file_path):
    data = {
    'fl': np.nan,
    'fr': np.nan,
    'rl': np.nan,
    'facecount': np.nan,
    'error':0
    }
    
    cap = cv2.VideoCapture(file_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    global pulses_forehead,pulses_leftc,pulses_rightc
    
    pulses_forehead=[]
    pulses_leftc=[]
    pulses_rightc=[]
    
    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
    
        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = detector(gray)
        
        if(len(faces) > 1):
            data['facecount'] = len(faces)
            return data
            break
    
        # Process each detected face
        for face in faces:
            # Detect facial landmarks
            landmarks = predictor(gray, face)
            
            # Identify forehead region based on landmarks
            forehead_r1=[0,0,0,0] #x1,x2,y1,y2
            
            forehead_r1[0] = landmarks.part(21).x # Left eyebrow-x
            forehead_r1[1] = landmarks.part(22).x  # Right eyebrow-x
            forehead_r1_height=int((forehead_r1[1] - forehead_r1[0]))
            forehead_r1[2] = landmarks.part(21).y - forehead_r1_height # Left eyebrow-y
            forehead_r1[3] = landmarks.part(22).y # Right eyebrow-y
            
            # Identify left cheek region-1 based on landmarks
            leftc_r1=[0,0,0,0] #x1,x2,y1,y2
            
            leftc_r1[0]=landmarks.part(41).x
            leftc_r1[1]=landmarks.part(39).x
            leftc_r1[2]=landmarks.part(28).y
            leftc_r1[3]=landmarks.part(30).y
    
            # Identify right cheek region-1 based on landmarks
            rightc_r1=[0,0,0,0] #x1,x2,y1,y2
            
            rightc_r1[0]=landmarks.part(42).x
            rightc_r1[1]=landmarks.part(46).x
            rightc_r1[2]=landmarks.part(28).y
            rightc_r1[3]=landmarks.part(30).y
            
            try:
                calculate_pulse_signal(frame[forehead_r1[2]:forehead_r1[3],forehead_r1[0]:forehead_r1[1]],pulses_forehead)
                calculate_pulse_signal(frame[leftc_r1[2]:leftc_r1[3],leftc_r1[0]:leftc_r1[1]],pulses_leftc)
                calculate_pulse_signal(frame[rightc_r1[2]:rightc_r1[3],rightc_r1[0]:rightc_r1[1]],pulses_rightc)   
            except Exception as e:
                print(e,file_path)
                data['error']=1
                return data
    
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # handle if q pressed
            break
    # Release video capture
    cap.release()

    # data = {'full_face': pulses_fullf, 'forehead': pulses_forehead, 'left_cheek': pulses_leftc, 'right_cheek': pulses_rightc}
    datadf = {'forehead': pulses_forehead, 'left_cheek': pulses_leftc, 'right_cheek': pulses_rightc}
    df = pd.DataFrame(datadf)
    correlation_matrix = df.corr()
    
    data = {
    'fl': correlation_matrix['forehead']['left_cheek'],
    'fr': correlation_matrix['forehead']['right_cheek'],
    'rl': correlation_matrix['right_cheek']['left_cheek'],
    'facecount': 1,
    'error':0
    }
    
    return data


In [27]:
excel_file = 'celebDF_DS.xlsx'  # Replace with the path to your Excel file
output_file='celebDF_DS_Corr2.xlsx'
results=[]

# Read the existing Excel file    
existing_df = pd.read_excel(excel_file)
count=0
for filepath in existing_df['filepath']:
    count+=1
    if(count>=160):
        print(f"processing: {filepath}")
        result = calculateResult(filepath)
        results.append(result)
# updating Results DataSet
print(results)
try:
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file,index=False)
    # updated_df = pd.concat([existing_df, results_df], axis=1, ignore_index=True)
    # updated_df.to_excel('celebDF_DS_Corr_combined1.xlsx', index=False)
except Exception as e:
    print(f"An error occurred: {e}")

processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0001.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0002.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0003.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0004.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0005.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0006.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0007.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0008.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id16_0009.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id17_0000.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id17_0001.mp4
processing: C:/Users/dpava/Desktop/celeb_df_v1/Celeb-synthesis\id0_id17_0002.mp4
processing: C:/Users/dpava/D

In [28]:
# df2 = pd.read_excel("celebDF_DS_Corr1.xlsx")
# df1 = pd.read_excel("celebDF_DS.xlsx")
# combined_df = pd.concat([df1,df2],axis=1, ignore_index=True)    
# combined_df.to_excel('celebDF_DS_Corr_combined1.xlsx', index=False)

In [29]:
print(results)

[{'fl': 0.0377298767340877, 'fr': -0.06369821047785719, 'rl': 0.13490282312447865, 'facecount': 1, 'error': 0}, {'fl': 0.07835702517642269, 'fr': -0.23444309675528996, 'rl': 0.068644054768274, 'facecount': 1, 'error': 0}, {'fl': -0.031194913001513024, 'fr': -0.02458652624237638, 'rl': 0.0451187339155603, 'facecount': 1, 'error': 0}, {'fl': -0.11165242737568064, 'fr': 0.009029002389239269, 'rl': 0.10432303095151614, 'facecount': 1, 'error': 0}, {'fl': 0.09794019141859608, 'fr': 0.13135577129016923, 'rl': 0.26467326344535347, 'facecount': 1, 'error': 0}, {'fl': 0.14746464454449817, 'fr': -0.008727994032389818, 'rl': 0.1359074743713416, 'facecount': 1, 'error': 0}, {'fl': -0.09332499883355658, 'fr': 0.032956582507776264, 'rl': 0.027040033548192486, 'facecount': 1, 'error': 0}, {'fl': nan, 'fr': nan, 'rl': nan, 'facecount': 2, 'error': 0}, {'fl': 0.08754292758969497, 'fr': 0.039345694644012136, 'rl': -0.06839556576377354, 'facecount': 1, 'error': 0}, {'fl': 0.09818481750934324, 'fr': -0.05

In [30]:
print('hi')

hi


In [31]:
print(results)

[{'fl': 0.0377298767340877, 'fr': -0.06369821047785719, 'rl': 0.13490282312447865, 'facecount': 1, 'error': 0}, {'fl': 0.07835702517642269, 'fr': -0.23444309675528996, 'rl': 0.068644054768274, 'facecount': 1, 'error': 0}, {'fl': -0.031194913001513024, 'fr': -0.02458652624237638, 'rl': 0.0451187339155603, 'facecount': 1, 'error': 0}, {'fl': -0.11165242737568064, 'fr': 0.009029002389239269, 'rl': 0.10432303095151614, 'facecount': 1, 'error': 0}, {'fl': 0.09794019141859608, 'fr': 0.13135577129016923, 'rl': 0.26467326344535347, 'facecount': 1, 'error': 0}, {'fl': 0.14746464454449817, 'fr': -0.008727994032389818, 'rl': 0.1359074743713416, 'facecount': 1, 'error': 0}, {'fl': -0.09332499883355658, 'fr': 0.032956582507776264, 'rl': 0.027040033548192486, 'facecount': 1, 'error': 0}, {'fl': nan, 'fr': nan, 'rl': nan, 'facecount': 2, 'error': 0}, {'fl': 0.08754292758969497, 'fr': 0.039345694644012136, 'rl': -0.06839556576377354, 'facecount': 1, 'error': 0}, {'fl': 0.09818481750934324, 'fr': -0.05